In [1]:
# Import necessary packages
from smtplib import SMTP

from urllib.request import urlopen, Request
from urllib.parse import quote
import random
import re

from chinese import ChineseAnalyzer

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
# Find list of songs
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

chart_html = urlopen(Request('https://kma.kkbox.com/charts/weekly/song?cate=297', headers=headers)).read()
songs = re.findall(r'"song_url":"https:\\/\\/www.kkbox.com\\/tw\\/tc\\/song\\/(\S{24}-index.html)"',
           chart_html.decode())

# Choose a random song out of the 50
i = random.randint(0, len(songs)-1)
song_link = 'https://www.kkbox.com/tw/tc/song/' + songs[i]
print(song_link)

https://www.kkbox.com/tw/tc/song/by.00WOSmSpknDuzknDuz0XL-index.html


In [8]:
# Find title of song and clean up a bit
song_html = urlopen(Request(song_link, headers=headers)).read()
title = re.findall(r'\<title\>([^\t\n\r\f\v]*)-歌詞', song_html.decode())[0] \
          .replace('&lt;', '<').replace('&gt;', '>')
print(title)

# Find artist name
artist = re.findall(r'\<title\>[^\t\n\r\f\v]*-歌詞-([^\t\n\r\f\v]*)-', song_html.decode())[0] \
          .replace('&lt;', '<').replace('&gt;', '>')
print(artist)

# Find lyrics and clean up a bit
lyrics = re.findall(r'"text":([^\t\n\r\f\v}]*)', song_html.decode())[0] \
           .replace('\\n', '\n').replace('"', '').replace('\\r', '')

CHANGE
瘦子E.SO


In [5]:
# Get video link from youtube
html = urlopen("https://www.youtube.com/results?search_query=%s"%(quote(title + artist))).read()
video_ids = re.findall(r"watch\?v=(\S{11})", html.decode())
video_link = "https://www.youtube.com/watch?v=" + video_ids[0]
print(video_link)

https://www.youtube.com/watch?v=HTRQ0n4yjfs


In [6]:
# Translate lyrics
analyzer = ChineseAnalyzer()
result = analyzer.parse(lyrics, traditional=True)

# Create list of unique words
seen = set()
words = [word for word in result.tokens() if not (word in seen or seen.add(word))]

# Update word list by appending pinyin and translation(s) to each word
translations = []
for word in words:
    defs = result[word]
    
    # Only proceed if matches to a word with a definition and pinyin
    if defs[0].pinyin != None:
        
        # Go through all definitions if multiple (since dictionary need a separate key for each)
        for d in defs:
            translations.append(word + ': ' + ''.join(d.pinyin) + ', ' + ', '.join(d.definitions))
            
# Make translations into one giant block of text
translations = '\n'.join(translations)

In [2]:
# Set up the SMTP server
s = SMTP(host='smtp.gmail.com', port=587)
s.starttls()
s.login('XXX@gmail.com', 'pwdstring')

(235, b'2.7.0 Accepted')

In [7]:
# Create message
msg = MIMEMultipart()

# https://www.freecodecamp.org/news/send-emails-using-code-4fcea9df63f/
message = 'Song of the day: {}\n'.format(title) + \
          'Artist: {}\n\n'.format(artist) + \
          'Song link: {}\n'.format(song_link) + \
          'Video link: {}\n\n'.format(video_link) + \
          'Lyrics: {}\n\n'.format(lyrics) + \
          'Translations: \n{}'.format(translations)

# Set up the parameters of the message
msg['To'] = 'XXX@gmail.com'
msg['Subject'] = "Daily digest"

# Add in the message body
msg.attach(MIMEText(message, 'plain'))

# Send the message
s.send_message(msg)
del msg

# Terminate the SMTP session and close the connection
s.quit()

(221, b'2.0.0 closing connection c1sm401478qta.86 - gsmtp')